# network.py

In [1]:
import random 
import numpy as np
class Network1(object):
    
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
        
    def feedforword(self, a):
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k + mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} complete".format(j))
        


    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # backward pass 
        delta = self.cost_derivative(activations[-1], y) * \
                sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta 
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        
        return (nabla_b, nabla_w)
    
    def evaluate(self,test_data):
        
        test_result = [(np.argmax(self.feedforword(x)), y)
                      for (x,y) in test_data]
        return sum(int(x==y) for (x,y) in test_result)
    
    def cost_derivative(self, output_activations,y):
        return (output_activations - y)
    
def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))

# mnist_loader.py

In [1]:
# %load mnist_loader.py
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [3]:
`tr_d, va_d, te_d = load_data()

# test

In [3]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

In [4]:
net = Network1([784, 30, 10])

In [5]:
net.SGD(training_data, 1, 10, 3.0, test_data=test_data)


Epoch 0 : 9031 / 10000


In [8]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
import network2
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
net.large_weight_initializer()
net.SGD(training_data, 10, 10, 0.5, evaluation_data=test_data,
        monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9145 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 9273 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 9349 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9394 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9417 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9467 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9465 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9454 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9488 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9459 / 10000


([], [9145, 9273, 9349, 9394, 9417, 9467, 9465, 9454, 9488, 9459], [], [])

# overfitting

In [1]:

import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
import network2

net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
net.large_weight_initializer()
net.SGD(list(training_data)[:1000], 100, 10, 0.5,  #list(training_data)[:1000]
    evaluation_data=test_data,
    lmbda = 0.1, # this is a regularization parameter
    monitor_evaluation_cost=True,
    monitor_evaluation_accuracy=True,
    monitor_training_cost=True,
    monitor_training_accuracy=True)

Epoch 0 training complete
Cost on training data: 1203.0090214904178
Accuracy on training data: 638 / 1000
Cost on evaluation data: 1203.4062991530584
Accuracy on evaluation data: 5329 / 10000
Epoch 1 training complete
Cost on training data: 1192.4582025147165
Accuracy on training data: 723 / 1000
Cost on evaluation data: 1192.985413664471
Accuracy on evaluation data: 5936 / 10000
Epoch 2 training complete
Cost on training data: 1182.3362117293307
Accuracy on training data: 837 / 1000
Cost on evaluation data: 1182.9081248020473
Accuracy on evaluation data: 6975 / 10000
Epoch 3 training complete
Cost on training data: 1172.7878897212183
Accuracy on training data: 867 / 1000
Cost on evaluation data: 1173.4323235306697
Accuracy on evaluation data: 7213 / 10000
Epoch 4 training complete
Cost on training data: 1163.543130540585
Accuracy on training data: 904 / 1000
Cost on evaluation data: 1164.2212612819574
Accuracy on evaluation data: 7466 / 10000
Epoch 5 training complete
Cost on training

Accuracy on evaluation data: 8210 / 10000
Epoch 43 training complete
Cost on training data: 841.3863603216412
Accuracy on training data: 999 / 1000
Cost on evaluation data: 842.5425609004092
Accuracy on evaluation data: 8215 / 10000
Epoch 44 training complete
Cost on training data: 834.1656690029707
Accuracy on training data: 999 / 1000
Cost on evaluation data: 835.3201247617696
Accuracy on evaluation data: 8217 / 10000
Epoch 45 training complete
Cost on training data: 827.0181541673811
Accuracy on training data: 999 / 1000
Cost on evaluation data: 828.1728889066409
Accuracy on evaluation data: 8210 / 10000
Epoch 46 training complete
Cost on training data: 819.9209271337588
Accuracy on training data: 999 / 1000
Cost on evaluation data: 821.0802181818418
Accuracy on evaluation data: 8213 / 10000
Epoch 47 training complete
Cost on training data: 812.8470099080919
Accuracy on training data: 999 / 1000
Cost on evaluation data: 813.9938741482866
Accuracy on evaluation data: 8242 / 10000
Epo

Accuracy on evaluation data: 8366 / 10000
Epoch 86 training complete
Cost on training data: 580.2600893019332
Accuracy on training data: 1000 / 1000
Cost on evaluation data: 581.3849466859306
Accuracy on evaluation data: 8363 / 10000
Epoch 87 training complete
Cost on training data: 575.2981506141005
Accuracy on training data: 1000 / 1000
Cost on evaluation data: 576.4168247030275
Accuracy on evaluation data: 8379 / 10000
Epoch 88 training complete
Cost on training data: 570.4083533707984
Accuracy on training data: 1000 / 1000
Cost on evaluation data: 571.5241873544395
Accuracy on evaluation data: 8384 / 10000
Epoch 89 training complete
Cost on training data: 565.5454827906468
Accuracy on training data: 1000 / 1000
Cost on evaluation data: 566.6609192559854
Accuracy on evaluation data: 8404 / 10000
Epoch 90 training complete
Cost on training data: 560.7386619122908
Accuracy on training data: 1000 / 1000
Cost on evaluation data: 561.8522309946244
Accuracy on evaluation data: 8385 / 1000

([1203.4062991530584,
  1192.985413664471,
  1182.9081248020473,
  1173.4323235306697,
  1164.2212612819574,
  1155.165712549201,
  1146.2867036503221,
  1137.2833110812892,
  1128.2588473238518,
  1119.4047314673658,
  1110.5355101167672,
  1101.609957639381,
  1092.8270754089729,
  1084.0371431365063,
  1075.2526908229027,
  1066.5870980774855,
  1057.889510794536,
  1049.2802243071455,
  1040.6797922046705,
  1032.0985690518812,
  1023.6026270906094,
  1015.120677864256,
  1006.7063537636782,
  998.3809419712885,
  990.0987387519884,
  981.8105416680027,
  973.6177212408392,
  965.4517666487301,
  957.3926199504664,
  949.3522467436115,
  941.3470458585681,
  933.4264645939137,
  925.5499291584887,
  917.7246925100491,
  909.9347081594067,
  902.2107310434396,
  894.5625241709088,
  886.9639638942211,
  879.4244219550106,
  871.9281173083243,
  864.5083913779064,
  857.1331739352325,
  849.7974511447944,
  842.5425609004092,
  835.3201247617696,
  828.1728889066409,
  821.0802181818

# network2.py

In [6]:
# Standard library
import json
import random
import sys

# Third-party libraries
import numpy as np


# 激活函数
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))


def sigmoid_prime(z):
    
    return sigmoid(z) * (1 - sigmoid(z))

# cost function
class QuadraticCost(object):
    
    @staticmethod
    def fn(a, y):
        
        return 0.5*np.linalg.norm(a - y) ** 2
    
    @staticmethod
    def delta(z, a, y):
        return (a - y)*sigmoid_prime(z)
    
class CrossEntropyCost(object):
    
    @staticmethod
    def fn(a, y):
        
        return np.sum(np.nan_to_num(-y * np.log(a) - (1-y))*np.log(1 - a))
    
    @staticmethod
    def delta(z, a, y):
        return (a - y)


# Main Network Class
class Network2(object):
    
    def __init__(self, sizes, cost = CrossEntropyCost):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost = cost
    
    def default_weight_initializer(self):
        
        self.biases = [np.random.randn(y,1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y,x) / np.sqrt(x)
                       for x, y in zip(self.sizes[:-1], self.sizes[1:])]
    
    def large_weight_initializer(self):

        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
        
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
           lmbda =0.0,
           evaluation_data = None,
           monitor_evaluation_cost = False,
           monitor_evaluation_accuracy = False,
           monitor_training_cost = False,
           monitor_training_accuracy = False,
           early_stopping_n = 0):
        
#         best_accuracy = 1
        
        training_data = list(training_data)
        n = len(training_data)
        
        if evaluation_data:
            evaluation_data = list(evaluation_data)
            n_data = len(evaluation_data)
        
        # early stopping functionality:
        best_accuracy = 0
        no_accuracy_change = 0
        
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                    training_data[k: k + mini_batch_size]
                    for k in range(0, n, mini_batch_size)]
            print("Epoch %s training complete" %j)

            if monitor_evaluation_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))

            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, n))

            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data))  

            # Early stopping:

            if early_stopping_n > 0:
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    no_accuracy_change = 0
                else:
                    no_accuracy_change +=1
                if (no_accuracy_change == early_stopping_n):
                    return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy
        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy


    def update_mini_batch(self, mini_batch, eta, lmbda, n):


        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1 - eta * (lmbda / n)) * w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]



    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

    
    def accuracy(self, data, convert=False):

        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                       for (x, y) in data]

        result_accuracy = sum(int(x == y) for (x, y) in results)
        return result_accuracy

    def total_cost(self, data, lmbda, convert=False):
        """Return the total cost for the data set ``data``.  The flag
        ``convert`` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the ``accuracy`` method, above.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y) / len(data)
            cost += 0.5 * (lmbda / len(data)) * sum(
                np.linalg.norm(w) ** 2 for w in self.weights)  # '**' - to the power of.
        return cost

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()


#### Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net


#### Miscellaneous functions
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the j'th position
    and zeroes elsewhere.  This is used to convert a digit (0...9)
    into a corresponding desired output from the neural network.

    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

- L2 regularization
$$
\begin{eqnarray} 
C&=&C_0 + \frac{\lambda}{2n} \sum_w w^2\\
w \rightarrow w' &= &w\left(1 - \frac{\eta \lambda}{n} \right)
  - \eta \frac{\partial C_0}{\partial w}.
\end{eqnarray}
$$

# test2

In [5]:
import numpy as np
a = 0.0
y = 0.0
np.sum(np.nan_to_num(-y * np.log(a) - (1 - y) * np.log(1 - a)))

<ipython-input-5-581789e6e931>:4: RuntimeWarning: divide by zero encountered in log
  np.sum(np.nan_to_num(-y * np.log(a) - (1 - y) * np.log(1 - a)))
<ipython-input-5-581789e6e931>:4: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(np.nan_to_num(-y * np.log(a) - (1 - y) * np.log(1 - a)))


0.0

In [9]:

import mnist_loader
training_data, validation_data, test_data = \
mnist_loader.load_data_wrapper()
import network2
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
net.SGD(training_data, 10, 10, 0.5,
        lmbda = 5.0,
        evaluation_data=validation_data,
        monitor_evaluation_accuracy=True,
        monitor_evaluation_cost=True,
        monitor_training_accuracy=True,
        monitor_training_cost=True)

Epoch 0 training complete
Cost on training data: 3940.07802519372
Accuracy on training data: 47050 / 50000
Cost on evaluation data: 3940.074064955377
Accuracy on evaluation data: 9432 / 10000
Epoch 1 training complete
Cost on training data: 4959.085340145698
Accuracy on training data: 47528 / 50000
Cost on evaluation data: 4959.0930469967125
Accuracy on evaluation data: 9505 / 10000
Epoch 2 training complete
Cost on training data: 5465.865270923692
Accuracy on training data: 47570 / 50000
Cost on evaluation data: 5465.8812296838005
Accuracy on evaluation data: 9489 / 10000
Epoch 3 training complete
Cost on training data: 5798.892292840804
Accuracy on training data: 47952 / 50000
Cost on evaluation data: 5798.912051636081
Accuracy on evaluation data: 9572 / 10000
Epoch 4 training complete
Cost on training data: 5991.915278811805
Accuracy on training data: 48050 / 50000
Cost on evaluation data: 5991.931314398319
Accuracy on evaluation data: 9581 / 10000
Epoch 5 training complete
Cost on 

([3940.074064955377,
  4959.0930469967125,
  5465.8812296838005,
  5798.912051636081,
  5991.931314398319,
  6087.08302080189,
  6238.681020397182,
  6326.3776089789735,
  6391.373205896668,
  6393.7774219747325],
 [9432, 9505, 9489, 9572, 9581, 9596, 9565, 9580, 9621, 9590],
 [3940.07802519372,
  4959.085340145698,
  5465.865270923692,
  5798.892292840804,
  5991.915278811805,
  6087.043429676771,
  6238.65300292026,
  6326.353278146069,
  6391.342880471811,
  6393.74456043591],
 [47050, 47528, 47570, 47952, 48050, 48175, 48050, 48161, 48287, 48360])

# network3.py

In [5]:
# - network3.py example:
import network3
from network3 import Network, ConvPoolLayer, FullyConnectedLayer, SoftmaxLayer # softmax plus log-likelihood cost is more common in modern image classification networks.

# read data:
training_data, validation_data, test_data = network3.load_data_shared()
# mini-batch size:
mini_batch_size = 10
net = Network([
    FullyConnectedLayer(n_in=784, n_out=100),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 10, mini_batch_size, 0.1, validation_data, test_data)


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 92.45%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.74%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 94.53%
This is the best validation accuracy to date.
The corresponding test accuracy is 93.72%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 95.64%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.02%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [6]:
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28),
                  filter_shape=(20, 1, 5, 5),
                  poolsize=(2, 2)),
    FullyConnectedLayer(n_in=20*12*12, n_out=100),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 10, mini_batch_size, 0.1, validation_data, test_data)

D:\Compiler\Anaconda3\Lib\site-packages\theano\tensor\nnet\conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000


KeyboardInterrupt: 